In [1]:
%matplotlib qt5

import sys
sys.path.append('src')

In [2]:
import matplotlib as mpl
dpi = 100
mpl.rcParams['figure.dpi']= dpi
mpl.rc("savefig", dpi=dpi)

In [3]:
from markov import *
import math

In [4]:
# values of kr and b to see different phenomena
# kr = 0.01, b = 20 to observe protein burst / rna more easily
# kr = 0.048, b = 10 to observe autoregulation (effects on transcription) more easily

# Motif 1: Expression from a single gene (the effects of low copy variation)

<img src="media/singlegene2.png" width="700">


## Kinetic Scheme

This image can be represented by the following scheme:

<img src="media/schematic.png" width="400">

We define the rates in the cell below. Note that

* `kr` represents $k_r$ and `gr` represents $\gamma_r$
* `kp` represents $k_p$ and `gp` represents $\gamma_p$
* `b` is definded as $\frac{k_p}{\gamma_r}$ and represents the average number of proteins translated during an mRNA lifetime.


<a id='rates1'></a>

## Slower transcription, higher burst

In [5]:
kr = 0.004  # rna/sec
gp = 1.0 / math.log(2)  # protein/hour
gp /= 60*60  # /sec
gr = 2 / math.log(2) # rna/min
gr /= 60

b = 120  # mean proteins made per rna
kp = b * gr

We are interested in tracking 3 quantities: DNA (`g`), RNA (`r`), and protein (`p`). We will place the initial quantities in a vector called `s0 = [g, r, p]`

In [6]:
s0 = np.array([1, 0, 0])

In the simple case there are 6 reactions whose rate is linear with the quantities `[g, r, p]`: DNA is created (doesn't happen in our setup), RNA is created, protein is created, DNA is degraded, RNA is degraded, or protein is degraded.

The first three linear reactions are captured by the matrix product `krates . s`, where `s` is the state vector with quantities `[g, r, p]`, while the latter three degradation reactions are captured by `grates . s`.

In [7]:
krates = [[ 0,  0,  0],
          [kr,  0,  0],
          [0,  kp,  0]]

grates = [[ 0,  0,  0],
          [ 0, gr,  0],
          [ 0,  0, gp]]

krates = np.array(krates)
grates = np.array(grates)

@numba.jit(nopython=True)
def Q(state):
    return np.concatenate((dot(krates, state), dot(grates, state)))


`Q` is a function that takes the current state of the system (`s = [g, r, p]`) and returns the current reaction rates for the various reactions.

In [8]:
Q(s0)

array([0.   , 0.004, 0.   , 0.   , 0.   , 0.   ])

`V` is an array that contains the "stoichometric change vectors" for our 6 reactions. Suppose the current state of the system has 1 gene (g), 1 mrna (r), and 300 protein (p), then `s = [1, 1, 300]` represents this information. If DNA is created, the change to the state vector is `s + [1, 0, 0]`, and we make it so that `V[0] = [1, 0, 0]`. So if "reaction 1" occurs then the state changes by `s + V[0]`, if reaction 2 occurs then the state changes by `s + V[1]`, etc.

In [9]:
d = len(krates)
V = np.vstack([np.eye(d), -np.eye(d)]).astype(np.int64)
V 

array([[ 1,  0,  0],
       [ 0,  1,  0],
       [ 0,  0,  1],
       [-1,  0,  0],
       [ 0, -1,  0],
       [ 0,  0, -1]])

## Problem 1

a. Why is the `krates` matrix off diagonal and the `grates` matrix diagonal?

b. Based off of the kinetic scheme above write down the differential equations for $\frac{dr}{dt}$ and $\frac{dp}{dt}$

c. Solve for the steady state concentrations $r_{\text{steady state}}$ and $p_{\text{steady state}}$

d. What is the expected $p_{\text{steady state}}$ given the rate constants [above](#rates1). Hint: it would probably be easiest to make another code and use the currently declared values for `kr, kp, gr, gp`.

d. Plotted below is $\frac{dp}{dt}$ vs. the amount of protein in the system (assuming $r_{\text{steady state}}$). Recall our analysis of fluctuations with the derivative of Gibbs free energy with system composition. Here we ask you to something similar. 

   * When the system has protein > $p_{\text{steady state}}$, what is the sign of the derivative $\frac{dp}{dt}$?
   * Given this sign, in a small time step, $dt$, will the system inrease or decrease the amount of protein?
   * Repeat the above analysis for when protein < $p_{\text{steady state}}$.
   * Would you conclude that this steady state is stable (most fluctuations are reflected back) or unstable (most fluctuations are amplified away from the steady state)?
    
e. Given your answers in (d), can we conclude that $p_{\text{steady state}} = \langle p \rangle$. Why or why not?

In [10]:
ps = np.linspace(0, 1500, 200)
dp = kp * kr / gr - gp * ps
fig, ax = plt.subplots(figsize=(5,4), dpi=100)
plt.plot(ps, dp)
plt.ylabel(r'$\frac{dp}{dt}$', rotation=0)
plt.xlabel('Amount of protein')
plt.hlines(0, 0, 1500, alpha=0.5)
plt.plot([1200], [0], marker='o', markersize=10, color="blue")

## The origin of noise in reaction systems comes from the random times at which individual reactions occur

We broadly assume that in an infinitesimal amount of time $dt$ that there are only 2 possibilities: no reaction occurs, or exactly 1 reaction occurs.

Consider a **single** reaction with the scheme:

$$ \text{reactants} \xrightarrow[]{k} \text{products} $$

Suppose with the current amounts of species in the system that this produces an **overall rate of reaction** $q$. For example, if $ A + B \xrightarrow[]{k} C $, then $q = k AB$. Under reasonable assumptions, we found in lecture that the probability that the reaction occurs at the particular time $\tau$ is 

$$ p(\tau) = q e^{-q \tau}$$

For multiple reactions, the probability of reaction $j$, occuring at time $\tau$ is:

$$ p(j, \tau) = q_j e^{-Q_0 \tau} $$

Where $Q_0 = \sum\limits_i q_i $ is the sum of the reaction rates.

Recall that our `Q` function returns `Q(state) = [rate of reaction 1, rate of reaction 2, etc. ]`.

Below is the function `simulate_cme` (simulate chemical master equation), which takes the arguments `Q, V, time, s0`. Based off of the initial rates in the system `Q(s0)`, it samples a random reaction, $j$, and a random "time until reaction", $\tau$, according to $ p(j, \tau) = q_j e^{-Q_0 \tau} $. Then it applies the appropropriate "stoichiometric change vector" to the current state of the system `s = s0 + V[j]`, and the new current time becomes `time = 0 + tau`.

It repeats this process to build up a reaction trajectory, a sequence of reactions that is consistent with the rate laws given by our kinetic scheme.


## Problem 2

a. Run the simulation in the two `code cells` below and plot the result. Save the overall plot. 

b. Zoom in on 4 events in which 1 mRNA was present. How much protein was produced during the lifetime of each of the 4 mRNA? What is the average? How does this correspond to our paramter `b`?

We have thus far described two situations with the same average, namely "slower rna, higher protein burst" or "faster rna, lower protein burst".

c. Go to the [bionumbers](https://bionumbers.hms.harvard.edu/search.aspx) website and enter "Estimated energy cost of RNA synthesis in adults". On average how many nucleotides are used to make an mRNA?

d. The final resulting mRNA (after processing) is on average ~1200 nucleotides. How many amino acids does this correspond to after translation? At the bionumbers website query "atp cost of protein synthesis". How many ATP does it take per amino acid? Which is more costly to produce the mRNA or the protein?

e. Which strategy "slower rna, higher protein burst" or "faster rna, lower protein burst" is more energy efficient? Breifly explain. Which strategy is more accurate (has lower variance)? Briefly explain.

In [11]:
time = 10000
tr1, ti1 = simulate_cme(Q, V, time, s0)
print(tr1.shape)
ti1 /= 60 # convert to minutes
protein1 = tr1[:, 2]
rna1 = tr1[:, 1]
dna1 = tr1[:, 0]

(6125, 3)


In [12]:
plt.step(ti1, protein1, label='protein', where='post')
plt.ylabel('Protein', color='C0')
plt.xlabel("Time (min)")
ax2 = plt.gca().twinx()
ax2.step(ti1, rna1, label='rna', color='C1', alpha=0.5, where='post')
ax2.set_ylabel("RNA", color='C1')
ax2.set_yticks(np.arange(tr1[:, 1].min(), tr1[:,1].max()+1))
print('')

<a id='nofeedback'></a>

## Faster transcription, lower burst

## Problem 3

a. Update the rates constants with the code cell below. Run the simulation, and plot the results. Save the figure.

b. What does the equilibrium value of protein appear to be from the plot? For this set of rate constants what is the analytic solution for the mean $\langle p \rangle$ (using 1c.). Discuss any discrepancy of the trajectory from the expected value.

c. Are the trajectories for this new set of rate constants more or less noisy than those from `problem 2`? Why or why not?

a. [How long](#time1) does it take to reach steady state?

In [13]:
kr = 0.048  # rna/sec
b = 10  # mean proteins made per rna
kp = b * gr

krates = [[ 0,  0,  0],
          [kr,  0,  0],
          [0,  kp,  0]]

grates = [[ 0,  0,  0],
          [ 0, gr,  0],
          [ 0,  0, gp]]

krates = np.array(krates)
grates = np.array(grates)

@numba.jit(nopython=True)
def Q(state):
    return np.concatenate((dot(krates, state), dot(grates, state)))

In [14]:
time = 15000
tr1, ti1 = simulate_cme(Q, V, time, s0)
print(tr1.shape)
ti1 /= 60 # convert to minutes
protein1 = tr1[:, 2]
rna1 = tr1[:, 1]
dna1 = tr1[:, 0]

(14872, 3)


In [15]:
plt.step(ti1, protein1, label='protein', where='post')
plt.ylabel('Protein', color='C0')
plt.xlabel("Time (min)")
ax2 = plt.gca().twinx()
ax2.step(ti1, rna1, label='rna', color='C1', alpha=0.5, where='post')
ax2.set_ylabel("RNA", color='C1')
ax2.set_yticks(np.arange(tr1[:, 1].min(), tr1[:,1].max()+1))
print('')

<a id='time1'></a>

### Time until steady state

We will use the time until within 1 standard devation of mean.

In [16]:
ps = protein1[-len(protein1)//5:]
m = ps.mean()
sd = ps.std()
ix = (protein1 > m - sd).argmax()
tss = ti1[ix]
print(f'mean: {m:.02f},  sd: {sd:.02f}')
print(f'time until steady state: {tss/60:.02f} hours')

mean: 1173.91,  sd: 90.26
time until steady state: 1.47 hours


# Simulating a population of cells

We have seen that when a particular gene (g) is turned on, that the noisy process of transcription and translation results varying protein levels. To observe what would happen to a population of cells that begin transcription of a gene we will run the above simulation many times with a fixed end point to observe how the trajectories vary.

## Problem 4

a. Run the code cell below for `ncells = 100` and then plot and save the resulting histogram. Clearly 100 cells is insufficient, but it gives us an idea of the spread.

b. Repeat with `ncells = 5000`. This will take a while (~2-5 mins). Once it is completed, uncomment the command `np.save('single', p_final1)`, and run the code cell to save the results to a file called `single.npy`. Re-comment the line (add a `#` at the beginning) to prevent accidentally overwritting the data. If you ever want to recover the data you can uncomment and run the line `p_final1 = np.load('single.npy')`. Plot the results of `ncells = 5000` and save the figure.

c. What is the mean and standard deviation of protein copy number across cells? How does the mean compare to your analytic result?

In [17]:
# this will take a while
ncells = 5000
p_final1 = []
for _ in range(ncells):
    tr, ti = runtime_cme(Q, V, time, s0)
    p_final1.append(tr[2])
p_final1 = np.array(p_final1)

In [18]:
# np.save('single', p_final1)

In [20]:
# p_final1 = np.load('data/single.npy')

In [19]:
print(f'Mean: {p_final1.mean()}, Standard deviation: {p_final1.std()}')

Mean: 1196.1008, Standard deviation: 112.05762909931657


In [20]:
plt.figure(figsize=(5,5))
plt.hist(p_final1, bins=np.arange(p_final1.min(), p_final1.max(), 20))
plt.xlabel('protein copy number')
plt.ylabel('Number of cells')
plt.tight_layout()

# Motif 2: Negative autoregulation

<img src="media/negautoregulation.png" width="700">

We will now introduce negative autoregulation into the system. To do this, we make a new function `Q_autoregulation`. This function will alter the transcription rate from the default transcription rate to a new one based on the current protein level.

Recall that for the standard ligand binding isotherm 

$$f_{\text{bound}} = \frac{1}{\frac{Kd}{[L]} + 1}$$

That a 100-fold change in binding occurs over the 100-fold change in ligand concentration from $[L] = 0.1 K_d$ to $[L] = 10 K_d$ (see Figure 12.11A in the book).

More generally, it is found that a 100-fold change in *response* (in this case binding fraction of ligand) can occur over much smaller ranges. This can arise if there are multiple binding sites available and if the binding of one ligand increases the likelihood of a subsequent ligand to bind. (See section 14.2 and 14.3 in the book). It can also arise in other ways. But the take home message is that **the sharpness of response** is a parameter that varies from system to system.

Generally, the response function of a system that is *saturable* is:

$$ y = \frac{1}{x^n + 1} $$

Where at high $x, y=1$ and at low $x, y=0$ and $n$ tunes the *sharpness* of response. 

Below we use the function

$$ \frac{k_r}{k_r^{max}} = \frac{1}{\left( \frac{K_d}{p} \right)^n + 1} $$

To tune the transcription rate based on the current protein copy number. Once protein reaches $p \approx Kd$ then the transcription rate is halved.


In [21]:
Kdr = 600  # protein copy number
h_coeffr = 1.5

@numba.jit(nopython=True)
def Q_autoregulation(state):
    p = dot(krates, state)
    y = (state[2]/Kdr)**h_coeffr
    p[1] *= 1 / (1 + y)
    m = dot(grates, state)
    rates = np.concatenate((p, m))
    assert np.all(rates >= 0), "negative rate encountered"
    return rates

<a id='p5'></a>

## Problem 5

a. Using the values `Kd = 600` and `n = 1.5`, calculate the value of $\frac{k_r}{k_r^{max}}$ for `p = 300, 600, 900`.

b. Run the simulation below, then plot and save the result. What value does the protein copy number appear to equilibrate to? Breifly explain why.

c. [How long](#time2) does it take for the system to approach steady state with negative feedback? Is this faster or slower than [before](#time1)? Explain.

In [22]:
tr2, ti2 = simulate_cme(Q_autoregulation, V, time, s0)
print(tr2.shape)
ti2 /= 60 # convert to minutes
protein2 = tr2[:, 2]
rna2 = tr2[:, 1]
dna2 = tr2[:, 0]

(8384, 3)


In [23]:
plt.step(ti2, protein2, label='protein', where='post')
plt.ylabel('Protein', color='C0')
plt.xlabel("Time (min)")
ax2 = plt.gca().twinx()
ax2.step(ti2, rna2, label='rna', color='C1', alpha=0.5, where='post')
ax2.set_ylabel("RNA", color='C1')
ax2.set_yticks(np.arange(tr2[:, 1].min(), tr2[:,1].max()+1))
print('')

<a id='time2'></a>

## Time until system reaches steady state

In [24]:
ps = protein2[len(protein2)//2:]
m = ps.mean()
sd = ps.std()
ix = (protein2 > m - sd).argmax()
tss = ti2[ix]
print(f'mean: {m:.02f},  sd: {sd:.02f}')
print(f'time until steady state: {tss/60:.02f} hours')

mean: 665.17,  sd: 60.11
time until steady state: 1.42 hours


<a id='p6'></a>

## Problem 6

a. Run the code cell below for `ncells = 100` and then plot and save the resulting histogram. Clearly 100 cells is insufficient, but it gives us an idea of the spread.

b. Repeat with `ncells = 5000`. This will take a while (~3-7 mins). Once it is completed, uncomment the command `np.save('negative', p_final2)`, and run the code cell to save the results to a file called `negative.npy`. Re-comment the line (add a `#` at the beginning) to prevent accidentally overwritting the data. If you ever want to recover the data you can uncomment and run the line `p_final2 = np.load('negative.npy')`. Plot the results of `ncells = 5000` and save the figure.

c. What is the mean and standard deviation of protein copy number across cells? How does the mean compare to your analytic result for the [no feedback](#nofeedback) case?

In [25]:
ncells = 50
p_final2 = []
for _ in range(ncells):
    tr, ti = runtime_cme(Q_autoregulation, V, time, s0)
    p_final2.append(tr[2])
p_final2 = np.array(p_final2)

In [26]:
# np.save('negative', p_final2)

In [27]:
p_final2 = np.load('data/negative.npy')

In [28]:
print(f'Mean: {p_final2.mean()}, Standard deviation: {p_final2.std()}')

Mean: 599.483, Standard deviation: 61.90682442994471


In [29]:
plt.figure(figsize=(5,5))
plt.hist(p_final2, bins=np.arange(p_final2.min(), p_final2.max(), 20))
plt.xlabel('protein copy number')
plt.ylabel('Number of cells')
plt.tight_layout()

print()

## Problem 7

a. Plot the results of `p_final1` and `p_final2` on the same axis, along with the rate of transcription as a function of protein copy number. Save the figure.

b. Which system, repressed or unrepressed, exhibits greater noise? Give a brief explanaition.

In [30]:
pool = np.concatenate([p_final1, p_final2])
bins = np.arange(0, pool.max()*1.1, 20)
hill = 1/(1+(bins/Kdr)**h_coeffr)

In [34]:
fig, ax = plt.subplots(figsize=(5,4))
v2, _, _ = plt.hist(p_final2, bins=bins, alpha=0.6, label='repressed')
v, _, _ = plt.hist(p_final1, bins=bins, alpha=0.6, label='unrepressed')
plt.xlabel('protein copy number')
plt.ylabel('number of cells')
ax.legend()

ax2 = ax.twinx()
ax2.plot(bins, 100*hill, color='C2', label='hill')
ax2.set_ylabel('transcription rate %', color='C2')
plt.tight_layout()

print()

# Motif 3: bistability from positive autoregulation (autocatalysis)

<img src="media/posautoregulation.png" width="700">

Now we suppose that the protein is capable of increasing the rate of transcription (possibly by helping recruit the polymerase). 

We again introduce a saturable response function:

$$ \frac{k_r}{k_{r0}} = k_{r0}\left( 1 + \frac{2}{\left( \frac{K_d}{p} \right)^n + 1} \right)$$

So at high protein copy number (relative to $K_d$) the transcription rate is 3 times higher than the baseline rate $k_{r0}$.

We set $K_d = 1.7 \langle p \rangle_{\text{no feedback}}$ 

So if $p$ stays close to the mean then the positive feedback won't have much of an effect, but if $p$ has an abnormally high fluctuation, then it effectively turns on the positive feeback and the new transcription rate is up to 3 times higher than the baseline.

This creates two general states, one with the low positive feedback, and one with high positive feedback (3x $k_{r0}$) and it arises due to the sharp response function coupled with the already inherently noisy process of translation.

In [35]:
pmean = kr * kp / gp / gr
Kdp = pmean * 1.47
h_coeffp = 12
factor = 2

@numba.jit(nopython=True)
def Q_autocatalytic(state):
    p = dot(krates, state)
    y = (state[2]/Kdp)**h_coeffp
    p[1] += p[1]*factor * y / (1 + y)
    m = dot(grates, state)
    rates = np.concatenate((p, m))
    assert np.all(rates >= 0), "negative rate encountered"
    return rates


## Problem 8

a. Assuming that there was no feedback in the system, but that simply `kr = 3 * 0.0048`, what would the new steady state protein concentration be?

b. Run the `code cell` below 4-5 times. You should be able to identify trajectories that remain in the slow transcription state, and those that switch into a high transcription state. Save 2 plots, one that typifies the slow transcription state, and another that typifies a transition to the fast transcription state.

c. For systems that transition to the fast transcription state, how long (overall) does it take to reach the new steady state? How does this compare to [no feedback](#nofeedback) and [negative feedback](#p5)?

In [36]:
tr3, ti3 = simulate_cme(Q_autocatalytic, V, 3*time, s0)
print(tr2.shape)
ti3 /= 60 # convert to minutes
protein3 = tr3[:, 2]
rna3 = tr3[:, 1]
dna3 = tr3[:, 0]

ps = protein3[len(protein3)//2:]
m = ps.mean()
sd = ps.std()
ix = (protein3 > m - sd).argmax()
tss = ti3[ix]
print(f'mean: {m:.02f},  sd: {sd:.02f}')
print(f'time until steady state: {tss/60:.02f} hours')

plt.step(ti3, protein3, label='protein', where='post')
plt.ylabel('Protein', color='C0')
plt.xlabel("Time (min)")
ax2 = plt.gca().twinx()
ax2.step(ti3, rna3, label='rna', color='C1', alpha=0.5, where='post')
ax2.set_ylabel("RNA", color='C1')
ax2.set_yticks(np.arange(tr3[:, 1].min(), tr3[:,1].max()+1))
print('')

(8384, 3)
mean: 1191.65,  sd: 96.97
time until steady state: 0.81 hours



## Problem 9

a. Run the cell below with `ncells = 5000`. This will take a while (~10-20 mins). Once it is completed, uncomment the command `np.save('bistable', p_final3)`, and run the code cell to save the results to a file called `bistable.npy`. Re-comment the line (add a `#` at the beginning) to prevent accidentally overwritting the data. If you ever want to reload the data you can uncomment and run the line `p_final3 = np.load('bistable.npy')`. Plot the results of `ncells = 5000` and save the figure.

b. What appears to be the mean of the two states? How does this compare to the analytic results of [no feedback](#nofeedback)? (3b)

c. If we decreased the parameter `n` from 12 down to 6, making the transition to higher transcription less sharp, what effect would that have on the valley between the two states? Give justification for your hypothesis.

In [43]:
# this will take a while
ncells = 3000
p_final3 = []
for _ in range(ncells):
    tr, ti = runtime_cme(Q_autocatalytic, V, 3*time, s0)
    p_final3.append(tr[2])
p_final3 = np.array(p_final3)

In [38]:
# np.save('bistable', p_final3)

In [39]:
# p_final3 = np.load('data/bistable.npy')

In [39]:
print(f'Mean: {p_final3.mean()}, Standard deviation: {p_final3.std()}')

Mean: 1884.7, Standard deviation: 1123.1827144325184


In [40]:
plt.figure(figsize=(5,4))
plt.hist(p_final3, bins=np.arange(p_final3.min(), p_final3.max(), 50))
plt.xlabel('protein copy number')
plt.ylabel('Number of cells')
plt.tight_layout()
print()

## Problem 10

a. Plot the results of the `no feedback` system (`p_final1`) along with the results from the system with positive feedback `p_final3`. Also plot the transcription rate as a function of protein concentration. Save the result.

b. Rank the systems above (no feedback, negative feedback, positive feedback) from lowest noise to highest noise in protein copy number.


In [41]:
pool = np.concatenate([p_final1, p_final3])
bins = np.arange(pool.min()/2, pool.max(), 50)
y = (bins/Kdp)**h_coeffp
hill = y / (1 + y)

In [42]:
fig, ax = plt.subplots(figsize=(5,4))
v2, _, _ = plt.hist(p_final3, bins=bins, alpha=0.6, label='autocatalytic', density=1)
v, _, _ = plt.hist(p_final1, bins=bins, alpha=0.6, label='linear', density=1)
plt.xlabel('protein copy number')
plt.ylabel('probability')

ax2 = ax.twinx()
ax2.plot(bins, 300*hill, color='C2', label='hill')
ax2.set_ylabel('transcription rate %', color='C2')

ax.legend(loc=7)


plt.tight_layout()

print()